In [1]:
import pandas as pd
from utils import load_csv
import os
import pickle
import numpy as np
import pandas as pd
from numpy import nan as Nan
from numpy import inf as inf
from tqdm import notebook as tqdm
from scipy.sparse import csr_matrix
from sklearn.model_selection import cross_val_predict
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import SelectFromModel

from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from data_preprocessing import *
from scoring import bootstrap_roc_auc
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostRegressor, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
from tqdm import tqdm
import catboost

In [4]:
df = pd.read_csv('exported.predictions.Kahle2020.csv')
groups = load_csv(f"groups_and_oxi_states_5_frames/df_features_step_0.pkl")
df = df.merge(groups, left_on=['src_id'], right_on=['stru_id'])
df = df.drop(['starting_structure', 'src_database', 'src_id'], axis = 1)
df['group_id'] = 1
df['relevance'] = -1
df.sort_values(by='diffusion_mean_cm2_s', ascending=False, inplace = True)
len_A = len(df[df['group'] == 'group_A'])
len_B = len(df[df['group'] == 'group_B'])
len_C = len(df[df['group'] == 'group_C'])
len_E = len(df[df['group'] == 'group_E'])
df.loc[df['group'] == 'group_A', 'relevance'] = np.arange(len_A)
df.loc[df['group'] == 'group_B', 'relevance'] = len_A + np.arange(len_B)
df.loc[df['group'] == 'group_E', 'relevance'] = len_A+len_B + np.arange(len_E)
df.loc[df['group'] == 'group_C', 'relevance'] = len_A+len_B+len_E + np.arange(len_C)
df['relevance'] = df.shape[0] - df['relevance']
    

In [5]:
X, y = (df['barrier'].to_numpy().reshape(-1, 1), df['relevance'].astype(int).to_numpy())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    train_pool = catboost.Pool(X_scaled[train_index, :], y[train_index], group_id = df['group_id'][train_index].to_list())
    cb_ranker = catboost.CatBoostRanker(loss_function='PairLogitPairwise', verbose = False)
    cb_ranker.fit(train_pool)
    prediction = cb_ranker.predict(X_scaled[test_index])
    preds[test_index] = np.exp(prediction) / (1 + np.exp(prediction))
estimated_mean, estimated_std = bootstrap_roc_auc(1000, y, preds)
print(estimated_mean, estimated_std)

4it [03:35, 53.93s/it]

In [45]:
X, y = (df['fv_1p0_disconnected'].to_numpy().reshape(-1, 1), df['is_good'].astype(int).to_numpy())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    model = CatBoostClassifier(eval_metric='AUC', verbose = False, random_state = 42)
    model.fit(X_scaled[train_index, :], y[train_index])
    preds[test_index] = model.predict_proba(X_scaled[test_index])[:, 1]
estimated_mean, estimated_std = bootstrap_roc_auc(1000, y, preds)
print(estimated_mean, estimated_std)

116it [02:04,  1.08s/it]


0.7341029824964577 0.049085469809764394


In [47]:
X, y = (df['fv_2p0_disconnected'].to_numpy().reshape(-1, 1), df['is_good'].astype(int).to_numpy())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    model = CatBoostClassifier(eval_metric='AUC', verbose = False, random_state = 42)
    model.fit(X_scaled[train_index, :], y[train_index])
    preds[test_index] = model.predict_proba(X_scaled[test_index])[:, 1]
estimated_mean, estimated_std = bootstrap_roc_auc(1000, y, preds)
print(estimated_mean, estimated_std)

116it [02:05,  1.08s/it]


0.7751256623913735 0.04818280232694281


In [48]:
X, y = (df['fv_3p0_disconnected'].to_numpy().reshape(-1, 1), df['is_good'].astype(int).to_numpy())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    model = CatBoostClassifier(eval_metric='AUC', verbose = False, random_state = 42)
    model.fit(X_scaled[train_index, :], y[train_index])
    preds[test_index] = model.predict_proba(X_scaled[test_index])[:, 1]
estimated_mean, estimated_std = bootstrap_roc_auc(1000, y, preds)
print(estimated_mean, estimated_std)

116it [02:05,  1.08s/it]


0.7227192284433122 0.045017607602623445


In [49]:
X, y = (df['fv_4p0_disconnected'].to_numpy().reshape(-1, 1), df['is_good'].astype(int).to_numpy())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    model = CatBoostClassifier(eval_metric='AUC', verbose = False, random_state = 42)
    model.fit(X_scaled[train_index, :], y[train_index])
    preds[test_index] = model.predict_proba(X_scaled[test_index])[:, 1]
estimated_mean, estimated_std = bootstrap_roc_auc(1000, y, preds)
print(estimated_mean, estimated_std)

116it [02:05,  1.08s/it]


0.629968810206653 0.05458774950291143


In [50]:
X, y = (df['fv_1p0_connected'].to_numpy().reshape(-1, 1), df['is_good'].astype(int).to_numpy())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    model = CatBoostClassifier(eval_metric='AUC', verbose = False, random_state = 42)
    model.fit(X_scaled[train_index, :], y[train_index])
    preds[test_index] = model.predict_proba(X_scaled[test_index])[:, 1]
estimated_mean, estimated_std = bootstrap_roc_auc(1000, y, preds)
print(estimated_mean, estimated_std)

116it [02:05,  1.08s/it]


0.7522408883091427 0.04944344967156948


In [51]:
X, y = (df['fv_2p0_connected'].to_numpy().reshape(-1, 1), df['is_good'].astype(int).to_numpy())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    model = CatBoostClassifier(eval_metric='AUC', verbose = False, random_state = 42)
    model.fit(X_scaled[train_index, :], y[train_index])
    preds[test_index] = model.predict_proba(X_scaled[test_index])[:, 1]
estimated_mean, estimated_std = bootstrap_roc_auc(1000, y, preds)
print(estimated_mean, estimated_std)

116it [02:05,  1.08s/it]


0.767612822912814 0.04493416211182906


In [52]:
X, y = (df['fv_3p0_connected'].to_numpy().reshape(-1, 1), df['is_good'].astype(int).to_numpy())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    model = CatBoostClassifier(eval_metric='AUC', verbose = False, random_state = 42)
    model.fit(X_scaled[train_index, :], y[train_index])
    preds[test_index] = model.predict_proba(X_scaled[test_index])[:, 1]
estimated_mean, estimated_std = bootstrap_roc_auc(1000, y, preds)
print(estimated_mean, estimated_std)

116it [02:05,  1.09s/it]


0.7505651982254903 0.04534117335416335


In [53]:
X, y = (df['fv_4p0_connected'].to_numpy().reshape(-1, 1), df['is_good'].astype(int).to_numpy())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    model = CatBoostClassifier(eval_metric='AUC', verbose = False, random_state = 42)
    model.fit(X_scaled[train_index, :], y[train_index])
    preds[test_index] = model.predict_proba(X_scaled[test_index])[:, 1]
estimated_mean, estimated_std = bootstrap_roc_auc(1000, y, preds)
print(estimated_mean, estimated_std)

116it [02:05,  1.09s/it]


0.7438843816103258 0.046370987380953126


In [67]:
X, y = df[['fv_1p0_disconnected',
 'fv_1p0_connected',
 'fv_1p0_WARNlowPES',
 'fv_2p0_disconnected',
 'fv_2p0_connected',
 'fv_2p0_WARNlowPES',
 'fv_3p0_disconnected',
 'fv_3p0_connected',
 'fv_3p0_WARNlowPES',
 'fv_4p0_disconnected',
 'fv_4p0_connected',
 'fv_4p0_WARNlowPES',
 'barrier']].to_numpy(), df['is_good'].astype(int).to_numpy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    model = CatBoostClassifier(eval_metric='AUC', verbose = False, random_state = 42)
    model.fit(X_scaled[train_index, :], y[train_index])
    preds[test_index] = model.predict_proba(X_scaled[test_index])[:, 1]
estimated_mean, estimated_std = bootstrap_roc_auc(1000, y, preds)
print(estimated_mean, estimated_std)

0it [00:00, ?it/s]

116it [02:27,  1.27s/it]


0.7868321916888675 0.04268185522768489


In [12]:
X, y = df.drop(['stru_label', 'stru_id', 'is_good'], axis = 1).to_numpy(), df['is_good'].astype(int).to_numpy()


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    X_tr, X_te, y_tr, y_te = (X_scaled[train_index, :], X_scaled[test_index, :], y[train_index], y[test_index])
    model = CatBoostClassifier(eval_metric='AUC', verbose = False, random_state = 42)
    summary = model.select_features(
                X = X_tr,
                y=y_tr,
                features_for_select= np.arange(len(X_tr[0])),
                num_features_to_select=50,
                steps=6,
                algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
                shap_calc_type=EShapCalcType.Regular,
                train_final_model=True,
                logging_level='Silent',
                plot=False)
    preds[test_index] = model.predict_proba(X_scaled[test_index])[:, 1]
estimated_mean, estimated_std = bootstrap_roc_auc(1000, y, preds)
print(estimated_mean, estimated_std)


0it [00:00, ?it/s]

1it [02:25, 145.82s/it]

In [35]:
X, y = (df['barrier'].to_numpy().reshape(-1, 1), df['is_good'].astype(int).to_numpy())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [40]:
X, y = df.drop(['stru_label', 'stru_id', 'is_good'], axis = 1), df['is_good'].astype(int).to_numpy()


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    X_tr, X_te, y_tr, y_te = (X_scaled[train_index, :], X_scaled[test_index, :], y[train_index], y[test_index])
    model = CatBoostClassifier(eval_metric='AUC', verbose = False, random_state = 42)
    summary = model.select_features(
                X = X_tr,
                y=y_tr,
                features_for_select= np.arange(len(X_tr[0])),
                num_features_to_select=50,
                steps=1,
                algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
                shap_calc_type=EShapCalcType.Regular,
                train_final_model=True,
                logging_level='Silent',
                plot=False)
    preds[test_index] = model.predict_proba(X_scaled[test_index])[:, 1]
    break
estimated_mean, estimated_std = bootstrap_roc_auc(1000, y, preds)
print(estimated_mean, estimated_std)


0it [00:00, ?it/s]

0it [01:48, ?it/s]


0.4932390480049693 0.007120986923703665


In [51]:
print(len(model.get_feature_importance()))

680


In [54]:
importance = np.zeros(X.shape[1])

In [55]:
importance[summary['selected_features']] += 1

In [ ]:
'fv_1p0_WARNlowPES', 'fv_2p0_WARNlowPES', 'fv_3p0_WARNlowPES', 'fv_4p0_WARNlowPES'

In [61]:
X.columns.to_list()

['fv_1p0_disconnected',
 'fv_1p0_connected',
 'fv_1p0_WARNlowPES',
 'fv_2p0_disconnected',
 'fv_2p0_connected',
 'fv_2p0_WARNlowPES',
 'fv_3p0_disconnected',
 'fv_3p0_connected',
 'fv_3p0_WARNlowPES',
 'fv_4p0_disconnected',
 'fv_4p0_connected',
 'fv_4p0_WARNlowPES',
 'barrier',
 '0_ape_features_mode-structure_CAMN.npy',
 '1_ape_features_mode-structure_CAMN.npy',
 '2_ape_features_mode-structure_CAMN.npy',
 '3_ape_features_mode-structure_CAMN.npy',
 '4_ape_features_mode-structure_CAMN.npy',
 '5_bc_features_mode-structure_CAMN.npy',
 '11_bf_features_mode-structure_CAMN.npy',
 '12_bf_features_mode-structure_CAMN.npy',
 '13_bf_features_mode-structure_CAMN.npy',
 '19_bf_features_mode-structure_CAMN.npy',
 '20_bf_features_mode-structure_CAMN.npy',
 '21_bf_features_mode-structure_CAMN.npy',
 '26_bf_features_mode-structure_CAMN.npy',
 '27_bf_features_mode-structure_CAMN.npy',
 '28_bf_features_mode-structure_CAMN.npy',
 '31_bf_features_mode-structure_CAMN.npy',
 '32_bf_features_mode-structure_C

In [59]:
with open("new_features_result/feature_names.txt", "w") as output:
        output.write(str(X.columns.to_list()))